In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math

### Monte Carlo Simulation Parameters

In [ ]:
# Input parameters

T = 25                                              #Reptition period (ns)
taus = np.arange(0.2, 15, 0.4)                      #Lifetimes (ns)
tns = 2**(np.arange(9)+2)                           #Time-bins
irfSigs = np.array((0.01, 0.1, 0.25, 0.5, 1, 2))    #IRF Standard Deviations (ns)

iterats = 5000                                      #Repetitions of simulations
numPhots = 75000                                    #Number of photons per measurement

np.set_printoptions(precision=2)
print("Simulation Parameters\n")
print("Taus(ns):\n{}\n".format(taus))
print("Time Bins:\n{}\n".format(tns))
print("IRF Sigma (ns):\n{}\n".format(irfSigs))

### Monte Carlo Simulation - Dirac

In [ ]:
# Dirac IRF
tn = 0
tau = 0
taufits = np.zeros((iterats, taus.size, tns.size))
intens = np.zeros((iterats, taus.size, tns.size))

def expofunc(x, par):
    return np.exp(-T*x/(tn*par))*(np.exp(T/(tn*par))-1)/(1-np.exp(-T/par))

# Generate data and fit
for tnidx in range(tns.size):
    tn =tns[tnidx]
    ti = np.arange(tn)+1
    t = ti*(T/tn)
    
    for tauidx in range(taus.size):
        tau = taus[tauidx]
        print("tn: {}  tau: {:.2f}".format(tn, tau))
        pdf = expofunc(ti, tau)
        rng = np.random.default_rng()
        decay = rng.poisson(pdf*numPhots, (iterats, tn))

        intens[:,tauidx,tnidx] = np.sum(decay, 1)

        for n in range(iterats):
            trace = decay[n,:];
            init_val = sum(trace * t)/sum(trace)
            fit_result = curve_fit(expofunc, ti, trace/sum(trace), p0=init_val, sigma=np.sqrt(trace)+1, maxfev = 50000, ftol=1e-10, xtol=1e-10, gtol=1e-10)
            taufits[n, tauidx, tnidx] = fit_result[0];

# Calculate F
tau_avg = np.mean(taufits,0)
tau_sd = np.std(taufits,0)
intens_avg = np.mean(intens,0)

F_dirac = tau_sd*np.sqrt(intens_avg)/tau_avg

# Save variables
np.save('F_dirac.npy', F_dirac)
#np.save('taufits_dirac.npy', taufits)

### Monte Carlo Simulation - Gaussian

In [ ]:
# With IRF
tn = 0
t0 = 0
tau = 0
irfSig = 0
taufits_unknown = np.zeros((iterats, taus.size, tns.size, irfSigs.size))
taufits_known = np.zeros((iterats, taus.size, tns.size, irfSigs.size))
Afits = np.zeros((iterats, taus.size, tns.size, irfSigs.size))
intens = np.zeros((iterats, taus.size, tns.size, irfSigs.size))

def expofunc(x, par):
    return np.exp(-T*x/(tn*par))*(np.exp(T/(tn*par))-1)/(1-np.exp(-T/par))

def irfgaus(x):
    normfact = sum((1/np.sqrt(2*math.pi*(irfSig/twidth)**2))*np.exp(-(x-t0)**2/(2*(irfSig/twidth)**2)))
    return (1/np.sqrt(2*math.pi*(irfSig/twidth)**2))*np.exp(-(x-t0)**2/(2*(irfSig/twidth)**2))/normfact

def expogausfunc(x, par):
    pdf = np.convolve(expofunc(x,par),irfgaus(x),mode ='full')
    pdf = pdf[(t0-1):(3*t0-1)]
    pdf_normfact = sum(pdf)
    return pdf/pdf_normfact

def irfgaus_unk(x, sig):
    normfact = sum((1/np.sqrt(2*math.pi*(sig/twidth)**2))*np.exp(-(x-t0)**2/(2*(sig/twidth)**2)))
    return (1/np.sqrt(2*math.pi*(sig/twidth)**2))*np.exp(-(x-t0)**2/(2*(sig/twidth)**2))/normfact

def expogausfunc_unk(x, par, sig):
    pdf = np.convolve(expofunc(x,par),irfgaus_unk(x, sig),mode ='full')
    pdf = pdf[(t0-1):(3*t0-1)]
    pdf_normfact = sum(pdf)
    return pdf/pdf_normfact


# Generate data and fit
for tnidx in range(tns.size):
    if tns.size == 1:
        tn = tns
    else:
        tn = tns[tnidx]
    ti = np.arange(tn)+1
    twidth = T/tn
    t = ti*twidth
    t0=int(tn/2)
    
    for irfidx in range(irfSigs.size):
        irfSig = irfSigs[irfidx]

        for tauidx in range(taus.size):
            tau = taus[tauidx]
            print("tn: {}  irf: {:.2f}  tau: {:.2f}".format(tn, irfSig, tau))
            pdf = expogausfunc(ti, tau)
            rng = np.random.default_rng()
            decay = rng.poisson(pdf*numPhots, (iterats, tn))

            intens[:,tauidx,tnidx,irfidx] = np.sum(decay, 1)

            for n in range(iterats):
                try:
                    trace = decay[n,:];
                    init_val = sum(trace * t)/sum(trace)
                    fit_known = curve_fit(expogausfunc, ti, trace/sum(trace), p0=init_val, sigma=np.sqrt(trace)+1, maxfev = 100000, ftol=1e-10, xtol=1e-10, gtol=1e-10)
                    fit_unknown = curve_fit(expogausfunc_unk, ti, trace/sum(trace), p0=(init_val, twidth), sigma=np.sqrt(trace)+1, maxfev = 100000, ftol=1e-10, xtol=1e-10, gtol=1e-10)
                    taufits_known[n, tauidx, tnidx, irfidx] = fit_known[0];
                    taufits_unknown[n, tauidx, tnidx, irfidx] = fit_unknown[0][0];
                    Afits[n, tauidx, tnidx, irfidx] = fit_unknown[0][1];
                except RuntimeError as err:
                    print("Error at tn: {}  irf: {:.2f}  tau: {:.2f}  iter: {}".format(tn, irfSig, tau, n))

# Calculate F
tau_avg_unk = np.mean(taufits_unknown,0)
tau_avg_k = np.mean(taufits_known,0)

tau_sd_unk = np.std(taufits_unknown,0)
tau_sd_k = np.std(taufits_known,0)

intens_avg = np.mean(intens,0)

F_unknown = tau_sd_unk*np.sqrt(intens_avg)/tau_avg_unk
F_known = tau_sd_k*np.sqrt(intens_avg)/tau_avg_k

# Save variables
np.save('F_known.npy', F_known)
np.save('F_unknown.npy', F_unknown)
#np.save('taufits_known.npy', taufits_known)
#np.save('taufits_unknown.npy', taufits_unknown)

### Monte Carlo Simulation - Rectangular

In [ ]:
# Square wave
tn = 0
t0 = 0
tau = 0
irfSig = 0
taufits_sqrknown = np.zeros((iterats, taus.size, tns.size, irfSigs.size))
intens = np.zeros((iterats, taus.size, tns.size, irfSigs.size))

def expofunc(x, par):
    return np.exp(-T*x/(tn*par))*(np.exp(T/(tn*par))-1)/(1-np.exp(-T/par))

def irfsqr(x):
    normfact = sum(np.abs(x-t0)<=((2.355*irfSig)/(2*twidth)))
    return (np.abs(x-t0)<=((2.355*irfSig)/(2*twidth)))/normfact

def exposqrfunc(x, par):
    pdf = np.convolve(expofunc(x,par),irfsqr(x),mode ='full')
    pdf = pdf[(t0-1):(3*t0-1)]
    pdf_normfact = sum(pdf)
    return pdf/pdf_normfact


# Generate data and fit
for tnidx in range(tns.size):
    if tns.size == 1:
        tn = tns
    else:
        tn = tns[tnidx]
    ti = np.arange(tn)+1
    twidth = T/tn
    t = ti*twidth
    t0=int(tn/2)
    
    for irfidx in range(irfSigs.size):
        irfSig = irfSigs[irfidx]
        
        for tauidx in range(taus.size):
            tau = taus[tauidx]
            print("tn: {}  irf: {:.2f}  tau: {:.2f}".format(tn, irfSig, tau))
            pdf = exposqrfunc(ti, tau)
            rng = np.random.default_rng()
            decay = rng.poisson(pdf*numPhots, (iterats, tn))

            intens[:,tauidx,tnidx,irfidx] = np.sum(decay, 1)

            for n in range(iterats):
                try:
                    trace = decay[n,:];
                    init_val = sum(trace * t)/sum(trace)
                    fit_known = curve_fit(exposqrfunc, ti, trace/sum(trace), p0=init_val, sigma=np.sqrt(trace)+1, maxfev = 100000, ftol=1e-10, xtol=1e-10, gtol=1e-10)
                    taufits_sqrknown[n, tauidx, tnidx, irfidx] = fit_known[0];
                except RuntimeError as err:
                    print("Error at tn: {}  irf: {:.2f}  tau: {:.2f}  iter: {}".format(tn, irfSig, tau, n))

# Calculate F
tau_avg_k = np.mean(taufits_sqrknown,0)

tau_sd_k = np.std(taufits_sqrknown,0)

intens_avg = np.mean(intens,0)

F_knownsq = tau_sd_k*np.sqrt(intens_avg)/tau_avg_k

# Save variables
np.save('F_knownsq.npy', F_knownsq)
#np.save('taufits_sqrknown.npy', taufits_sqrknown)